In [40]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [41]:
df = pd.read_csv('../input/smile-twitter-emotion-dataset/smile-annotations-final.csv', names=['id', 'text', 'category'])
df.set_index('id', inplace=True)

In [42]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [43]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [44]:
df = df[~df.category.str.contains('\|')]


In [45]:
df = df[df.category != 'nocode']

In [46]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [47]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
df['label'] = lb.fit_transform(df.category)

In [48]:
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,2
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,2
614877582664835073,@Sofabsports thank you for following me back. ...,happy,2
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,2
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,2


In [49]:
df.category.unique()

array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [50]:
df.label.unique()

array([2, 3, 0, 1, 4, 5])

In [51]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.index.values,df.label.values,test_size=0.15,stratify=df.label.values,random_state=42)

In [52]:
df['data_type'] = ['not_set']*df.shape[0]

In [53]:
df.loc[X_train,'data_type'] = 'train'
df.loc[X_test, 'data_type'] = 'test'

In [54]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        0     test          9
                   train        48
disgust      1     test          1
                   train         5
happy        2     test        171
                   train       966
not-relevant 3     test         32
                   train       182
sad          4     test          5
                   train        27
surprise     5     test          5
                   train        30

In [55]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [56]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

In [57]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length = True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type=='test'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_mask_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_test = encoded_data_test['input_ids']
attention_mask_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df.data_type=='test'].label.values)

In [58]:
dataset_train = TensorDataset(input_ids_train, attention_mask_train, labels_train)
dataset_val = TensorDataset(input_ids_test, attention_mask_test, labels_test)

In [59]:
len(dataset_train),len(dataset_val)

(1258, 223)

In [60]:
from transformers import BertForSequenceClassification

In [61]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
num_labels=6,
output_attentions=False,
output_hidden_states=False)

In [62]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

In [63]:
batch_size = 15
dataloader_train = DataLoader(dataset_train,sampler=RandomSampler(dataset_train),batch_size=batch_size)
dataloader_test = DataLoader(dataset_val,sampler=RandomSampler(dataset_val),batch_size = batch_size)

In [64]:
from transformers import AdamW,get_linear_schedule_with_warmup

In [65]:
optimizer = AdamW(model.parameters(),lr=1e-5,eps=1e-8)

In [66]:
epochs = 10
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=len(dataloader_train)*epochs)

In [67]:
inv = lb.inverse_transform(df.label.unique())
inv


array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [68]:
import numpy as np
from sklearn.metrics import f1_score

In [69]:
def f1_score_func(preds,labels):
    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat,preds_flat,average='weighted')

In [80]:
def accuracy_per_class(preds,labels):
    inv = lb.inverse_transform(df.label.unique())
    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [71]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [72]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [75]:
def evaluate(dataloader_test):
    model.eval()
    loss_val_total = 0
    predictions,true_vals = [], []
    for batch in dataloader_test:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                  }
        with torch.no_grad():
            outputs = model(**inputs)
        
        loss =outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_test)
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [76]:
torch.cuda.empty_cache()
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.562495639636403
Validation loss: 0.4786341696977615
F1 Score (Weighted): 0.8096272196589624



Epoch 2
Training loss: 0.3810587210492009
Validation loss: 0.4567965467770894
F1 Score (Weighted): 0.8257297261781567



Epoch 3
Training loss: 0.2691663270816207
Validation loss: 0.4267682204643885
F1 Score (Weighted): 0.8530985967851674



Epoch 4
Training loss: 0.2014843269384333
Validation loss: 0.4356493314107259
F1 Score (Weighted): 0.8613545236031083



Epoch 5
Training loss: 0.15055364588763387
Validation loss: 0.4632007535547018
F1 Score (Weighted): 0.8560927364802682



Epoch 6
Training loss: 0.11607645986424316
Validation loss: 0.48067581119636693
F1 Score (Weighted): 0.8744906579116901



Epoch 7
Training loss: 0.09433617398497604
Validation loss: 0.5144772828867038
F1 Score (Weighted): 0.8734691371848142



Epoch 8
Training loss: 0.07715373630413697
Validation loss: 0.5086085125183065
F1 Score (Weighted): 0.8675797810456563



Epoch 9
Training loss: 0.07268772399540813
Validation loss: 0.5169873279519379
F1 Score (Weighted): 0.8637698391191224



Epoch 10
Training loss: 0.06914013193454593
Validation loss: 0.527288124213616
F1 Score (Weighted): 0.8637698391191224



In [77]:
model.load_state_dict(
    torch.load(
        "./finetuned_BERT_epoch_6.model", 
        map_location = torch.device('cpu')
    )
)


<All keys matched successfully>

In [78]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [82]:
inv

array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [ ]:
np.argmax(predictions,axis=1).flatten()